#  赛题链接：[常规赛：图神经网络入门节点分类](https://aistudio.baidu.com/aistudio/competition/detail/59)

## （1）赛题介绍
图神经网络（Graph Neural Network）是一种专门处理图结构数据的神经网络，目前被广泛应用于推荐系统、金融风控、生物计算中。图神经网络的经典问题主要有三种，包括节点分类、连接预测和图分类三种。本次比赛是图神经网络7日打卡课程的大作业，主要让同学们熟悉如何图神经网络处理节点分类问题。

在过去的一个世纪里，科学出版物的数量每12年增加近一倍，对每一种出版物的主题及领域进行自动分类已成为当下十分重要的工作。本次任务的目标是预测未知论文的主题类别，如软件工程，人工智能，语言计算和操作系统等。比赛所选35个领域标签已得到论文作者和arXiv版主确认并标记。

本次比赛选用的数据集为arXiv论文引用网络——ogbn-arixv数据集的子集。ogbn-arixv数据集由大量的学术论文组成，论文之间的引用关系形成一张巨大的有向图，每一条有向边表示一篇论文引用另一篇论文，每一个节点提供100维简单的词向量作为节点特征。在论文引用网络中，我们已对训练集对应节点做了论文类别标注处理。本次任务希望参赛者通过已有的节点类别以及论文之间的引用关系，预测未知节点的论文类别。

## （2）数据描述
本次赛题数据集由学术网络图构成，该图会给出每个节点的特征，以及节点与节点间关系（训练集节点的标注结果已给出）。

### 数据集简介：
**1.学术网络图数据：**

该图包含1647958条有向边，130644个节点，参赛者报名成功后即可通过比赛数据集页面提供edges.csv以及feat.npy下载并读取数据。图上的每个节点代表一篇论文，论文从0开始编号；图上的每一条边包含两个编号，例如 3，4代表第3篇论文引用了第4篇论文。图构造可以参照AiStudio上提供的基线系统项目了解数据读取方法。

**2.训练集与测试集：**

训练集的标注数据有70235条，测试集的标注数据有37311条。训练数据给定了论文编号与类别，如3，15 代表编号为3的论文类别为15。测试集数据只提供论文编号，不提供论文类别，需要参赛者预测其类别。

具体数据介绍：

**1.图数据：**

feat.npy为Numpy格式存储的节点特征矩阵，Shape为(130644, 100)，即130644个节点，每个节点含100维特征。其中 feat.npy可以用numpy.load(“feat.npy”)读取。edges.csv用于标记论文引用关系，为无向图，且由两列组成，没有表头。
| 字段 | 说明 |
| -------- | -------- |
| 第一列 | 边的初始节点 |
| 第二列 | 边的终止节点 |

**2.训练集： train.csv**
| 字段 | 说明 |
| -------- | -------- |
| nid | 训练节点在图上的Id |
| label | 训练节点的标签（类别编号从0开始，共35个类别） |

**3.测试集： test.csv**
| 字段 | 说明 |
| -------- | -------- |
| nid | 测试节点在图上的Id |

## （3）代码参考及改动
代码主要是根据怀瑾大大的比赛经验和心得进行了微小改动。本次主要是将传统的UniMP改动成为一种双流UniMP模型，即包含两个分支，一个分支的use_label属性被设置为False，另一个为True（但是并没有针对改动的有效性进行验证，貌似和原始版本效果差不多，觉得主要还是调参的贡献比较大，UniMP效果真的是牛）（PS:checkpoints忘保存了，被后面的结果给覆盖了）。

（1） [常规赛：论文引用网络节点分类比赛经验及心得](https://aistudio.baidu.com/aistudio/projectdetail/1921004?channelType=0&channel=0)


## （4）提交内容与格式

最终提交的submission.csv 格式如下：
| 字段 | 说明 |
| -------- | -------- |
| nid | 测试集节点在图上的id |
| label | 测试集的节点类别 |

**特别提示：** 提交的结果必须是类别的id，不能是概率。

提交样例：
| nid | label |
| -------- | -------- |
| 2 | 34 |
| 3 | 1 |
| 4 | 15 |
| … | … |


**注意**：选手提交的csv文件的第一行为”nid,label”，顺序调转将视为无效答案。接下来每一行的两个数字依次表示测试集提供的节点id以及对应的预测论文类别，注意给出的预测类别id要属于给定范围值且必须是整数。提交的数据行数要与test.csv一致。

# 运行方式
本次基线基于飞桨PaddlePaddle 1.8.4版本，若本地运行则可能需要额外安装pgl、easydict、pandas等模块。

## 1.本地运行
下载左侧文件夹中的所有py文件（包括build_model.py, model.py）,以及work目录，然后在右上角“文件”->“导出Notebook到py”，这样可以保证代码是最新版本），执行导出的py文件即可。完成后下载submission.csv提交结果即可。

## 2.AI Studio (Notebook)运行
依次运行下方的cell，完成后下载submission.csv提交结果即可。**若运行时修改了cell，推荐在右上角重启执行器后再以此运行，避免因内存未清空而产生报错**。 Tips：若修改了左侧文件夹中数据，也需要重启执行器后才会加载新文件。


# 代码整体逻辑

1. 读取提供的数据集，包含构图以及读取节点特征（用户可自己改动边的构造方式）

2. 配置化生成模型，用户也可以根据教程进行图神经网络的实现。

3. 开始训练

4. 执行预测并产生结果文件

## （1）环境配置

该项目依赖飞桨paddlepaddle==1.8.4, 以及pgl==1.2.0。请按照版本号下载对应版本就可运行。

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
!mkdir /home/aistudio/external-libraries

!pip install pgl==1.2.0 -i https://mirror.baidu.com/pypi/simple
!pip install xgboost==1.1.0 easydict -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://mirror.baidu.com/pypi/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/75/6c/fb30ff1703846db8e8e511399e41123ac968dc29171f04e9f0b2585e40c9/xgboost-1.1.0-py3-none-manylinux2010_x86_64.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/58/4f/11f34cfc57ead25752a7992b069c36f5d18421958ebd6466ecd849aeaf86/scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5b/0d/de55834c5ea0dd287cb1cb156c8bc120af2863c36e4d49b4dc28f174e278/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.21.4 which is incompatible.


In [2]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [3]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd

## （2）图网络配置

这里已经有很多强大的模型配置，你可以尝试简单的改一下config的字段。
例如，换成GAT的配置：
```
config = {
    "model_name": "GAT",
    "num_layers":  1,
    "dropout": 0.5,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "edge_dropout": 0.00,
}
```
下面采用了双流UniMP的配置：

In [4]:
from easydict import EasyDict as edict

config = {
    "model_name": "TS_UniMP",
    "num_layers": 2,
    "hidden_size": 32,
    "heads": 8,
    "learning_rate": 0.001,
    "dropout": 0.4,
    "weight_decay": 0.0003,
    "edge_dropout": 0.3,
    "use_label_e": True
}


config = edict(config)

## （3）数据加载模块

这里主要是用于读取数据集，包括读取图数据构图，以及训练集的划分。

In [5]:
from collections import namedtuple

Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("work/feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})
    
    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    
    df = pd.read_csv("work/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    #调整好超参数后可以增大训练集的比例
    train_part = int(len(node_index) * 0.9)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("work/test.csv")["nid"].values
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    return dataset

In [6]:
dataset = load()

train_index = dataset.train_index
train_label = np.reshape(dataset.train_label, [-1 , 1])
train_index = np.expand_dims(train_index, -1)

val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")

## （4）组网模块

这里是组网模块，目前已经提供了一些预定义的模型，包括**GCN**, **GAT**, **APPNP**等。可以通过简单的配置，设定模型的层数，hidden_size等。你也可以深入到model.py里面，去奇思妙想，写自己的图神经网络。

In [ ]:
import pgl
import model
import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model

# 使用CPU
#place = fluid.CPUPlace()

# # 使用GPU
place = fluid.CUDAPlace(0)

train_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()
with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        gw, loss, acc, pred = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)

test_program = fluid.Program()
with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        _gw, v_loss, v_acc, v_pred = build_model(dataset,
            config=config,
            phase="test",
            main_prog=test_program)


test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)

In [ ]:
def output(feed_dict, test_index,test_label, test_program, use_label_e, train_index, train_label, epoch):
    feed_dict["node_index"] = np.array(test_index, dtype="int64")
    feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签

    if use_label_e:
        feed_dict['label_idx'] = np.array(train_index, dtype="int64")
        feed_dict['label'] = np.array(train_label, dtype="int64")

    test_prediction = exe.run(test_program,
                                feed=feed_dict,
                                fetch_list=[v_pred],
                                return_numpy=True)[0]
    submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
    submission.to_csv("submission"+str(epoch)+".csv", index=False)

## （5）开始训练过程

图神经网络采用FullBatch的训练方式，每一步训练就会把所有整张图训练样本全部训练一遍。

In [ ]:
import os
use_label_e = True
label_rate = 0.6
epoch = 2000
exe.run(startup_program)
max_val_acc = 0

val_acc_history = 0
early_stop_counter = 0

# 这里可以恢复训练
pretrained = False
if pretrained:
    def name_filter(var):
        res = var.name in os.listdir('./output')
        return res
    fluid.io.load_vars(exe, './output',predicate=name_filter)

earlystop = 0
# 将图数据变成 feed_dict 用于传入Paddle Excecutor
feed_dict = gw.to_feed(dataset.graph)
for epoch in range(epoch):
    # Full Batch 训练
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    
    if use_label_e:
        train_idx_temp = np.array(train_index, dtype="int64")
        train_lab_temp = np.array(train_label, dtype="int64")
        state = np.random.get_state()
        np.random.shuffle(train_idx_temp)
        np.random.set_state(state)
        np.random.shuffle(train_lab_temp)

        label_idx=train_idx_temp[:int(label_rate*len(train_idx_temp))]
        unlabel_idx=train_idx_temp[int(label_rate*len(train_idx_temp)):]
        label=train_lab_temp[:int(label_rate*len(train_idx_temp))]
        unlabel=train_lab_temp[int(label_rate*len(train_idx_temp)):]

        feed_dict["node_index"] = unlabel_idx
        feed_dict["node_label"] = unlabel
        feed_dict['label_idx']= label_idx
        feed_dict['label']= label
    else:
        feed_dict["node_label"] = np.array(train_label, dtype="int64")
        feed_dict["node_index"] = np.array(train_index, dtype="int64")
        

    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)

    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid    
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    if use_label_e:
        feed_dict['label_idx'] = np.array(train_index, dtype="int64")
        feed_dict['label'] = np.array(train_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0])
    if val_acc[0] > max_val_acc:
        max_val_acc = val_acc[0]
        fluid.io.save_persistables(exe, './output', train_program)

    if val_acc_history <= val_acc[0]:
        val_acc_history = val_acc[0]
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter > 150:
            pass
            #break
    if  epoch == 1600 or epoch == 1800 or epoch == 1500 or epoch == 1300:
        output(feed_dict, test_index,test_label, test_program, use_label_e, train_index, train_label, epoch)

## （6）对测试集进行预测

训练完成后，我们对测试集进行预测。预测的时候，由于不知道测试集合的标签，我们随意给一些测试label。最终我们获得测试数据的预测结果。

In [ ]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签

if use_label_e:
    feed_dict['label_idx'] = np.array(train_index, dtype="int64")
    feed_dict['label'] = np.array(train_label, dtype="int64")

test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

## （7）生成提交文件

最后一步，我们可以使用pandas轻松生成提交文件，最后下载 submission.csv 提交就好了。

In [ ]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission.csv", index=False)

## 其他
关于UniMP的理解，以及改进trick可参考我的两篇分享：

[分享1](https://aistudio.baidu.com/aistudio/projectdetail/1650128)

[分享2](https://aistudio.baidu.com/aistudio/projectdetail/1462003)

## 赛题链接：[常规赛：图神经网络入门节点分类](https://aistudio.baidu.com/aistudio/competition/detail/59)


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 